# Vítej!

Tento dokument obsahuje zdrojové kódy animací k páté sérii seriálu KSP. Před spouštěním opět nezapomeň Manim importovat spuštěním následujícího řádku!

In [ ]:
from manim import *

## Vlastní animace

In [ ]:
%%manim -v WARNING -qh --disable_caching StarFox
# POZOR! Při změnách musíme opět zakázat cachování
# (Manim při změně animace nepozná změnu ve scéně)



class Roll(Animation):
    """Animace, která otočí objekt o daný úhel, trochu ho při tom zmenší a posune se do strany."""
    
    def __init__(self, mobject: Mobject, angle, direction, scale_ratio=0.85, **kwargs):
        """Konstruktor. Inicializuje potřebné věci animace."""
        # bude se nám hodit původní verze objektu, který animujeme
        # (v animaci ho otiž budeme měnit)
        self.original = mobject.copy()
        
        self.scale_ratio = scale_ratio
        self.angle = angle
        self.direction = direction
        
        super().__init__(mobject, **kwargs)

    def interpolate_mobject(self, alpha: float) -> None:
        """Funkce, která se volá každý snímek, aby se animace animovala."""
        
        # alpha je od 0 do 1, ale animace mohla jako parametr dostat rate funkci
        # proto je třeba tuto funkci na alphu aplikovat, a by se animace chovala správně
        actual_alpha = self.rate_func(alpha)

        # chceme, aby objekt měl na začátku scale 1, v půlce scale_ratio a na konci 1
        # tohle možná není nejelegantnější způsob, ale funguje
        scale_alpha = 1 - (1 - self.scale_ratio) * 2 * (0.5 - abs(actual_alpha - 0.5))
        
        # chceme, aby objekt měl na začátku startovní pozici, pak se posunul a nakonec se vrátil
        direction_alpha = there_and_back(actual_alpha)
        
        self.mobject.become(self.original.copy())\
            .rotate(actual_alpha * self.angle)\
            .scale(scale_alpha)\
            .shift(self.direction * direction_alpha)

        
class Dissolve(AnimationGroup):
    """Animace, která 'zmizí' objekt. Používáme zde AnimationGroup,
    jelikož animace mizení je tvořena dvěma různými animacemi."""
    
    def __init__(self, mobject: Mobject, **kwargs):
        """Konstruktor. Inicializuje potřebné věci animace."""
        self.original = mobject.copy()
        
        # způsob, jak do animate syntaxu dostaneme argumenty
        a1 = mobject.animate.scale(0)
        a2 = Flash(mobject, color=mobject.color)
        
        super().__init__(a1, a2, lag_ratio=0.75, **kwargs)
        
        
class StarFox(Scene):
    def construct(self):
        square = Square(color=BLUE, fill_opacity=0.75).scale(1.5)
        
        self.play(Roll(square, angle=PI, direction=LEFT * 0.75))
        self.play(Roll(square, angle=-PI, direction=RIGHT * 0.75))
        
        self.play(Dissolve(square))

## Pluginy

### Fyzika

In [ ]:
from manim_physics import *

In [ ]:
%%manim -v WARNING -qh FallingObjectsExample


# příklad z https://github.com/Matheart/manim-physics
# SpaceScene je třída podporující fyzikální interakce
class FallingObjectsExample(SpaceScene):
    def construct(self):
        circle = Circle().shift(UP)
        circle.set_fill(RED, 1)
        circle.shift(DOWN + RIGHT)

        rect = Square().shift(UP)
        rect.rotate(PI / 4)
        rect.set_fill(YELLOW_A, 1)
        rect.shift(UP * 2)
        rect.scale(0.5)

        ground = Line([-4, -3.5, 0], [4, -3.5, 0])
        wall1 = Line([-4, -3.5, 0], [-4, 3.5, 0])
        wall2 = Line([4, -3.5, 0], [4, 3.5, 0])
        walls = VGroup(ground, wall1, wall2)
        self.add(walls)

        self.play(
            DrawBorderThenFill(circle),
            DrawBorderThenFill(rect),
        )
        
        # až doposud se jednalo o normální Manim kód
        # nyní použijeme funkce, které objektům přidají fyziku
        self.make_rigid_body(rect, circle)  # čtverec a kruh jsou rigidní (hýbou se)
        self.make_static_body(walls)        # zdi jsou statické (nehýbou jse)
        
        # nyní počkáme - funkce výše přidaly objektům updatery
        self.wait(5)

In [ ]:
%%manim -v WARNING -qh ElectricFieldExample


# zde stačí Scene, protože používáme pouze nové objekty
# příklad upravený z https://github.com/Matheart/manim-physics
# POZOR: kód trvá postavit opravdu dlouho, doporučuji používat nižší kvalitu
class ElectricFieldExample(Scene):
    def construct(self):
        charge1 = Charge(-1, LEFT + DOWN)
        charge2 = Charge(2, RIGHT + DOWN)
        charge3 = Charge(-1, UP)
        
        def rebuild(field):
            """Funkce která přestaví elektrické pole."""
            field.become(ElectricField(charge1, charge2, charge3))
            
        field = ElectricField(charge1, charge2, charge3)
        
        self.add(field, charge1, charge2, charge3)
        
        self.play(Write(field), FadeIn(charge1), FadeIn(charge2), FadeIn(charge3))
        
        field.add_updater(rebuild)
        
        self.play(
            charge1.animate.shift(LEFT),
            charge2.animate.shift(RIGHT),
            charge3.animate.shift(DOWN * 0.5),
            run_time=2,
        )

In [ ]:
%%manim -v WARNING -qh MagnetismExample


# příklad upravený z https://github.com/Matheart/manim-physics
# OPĚT POZOR: kód trvá postavit opravdu dlouho, doporučuji používat nižší kvalitu
class MagnetismExample(Scene):
    def construct(self):
        current1 = Current(LEFT * 2.5)
        current2 = Current(RIGHT * 2.5, direction=IN)
        
        def rebuild(field):
            """Funkce která přestaví magnetické pole."""
            field.become(MagneticField(current1, current2))
                         
        field = MagneticField(current1, current2)
        
        self.play(Write(field), FadeIn(current1), FadeIn(current2))
        
        field.add_updater(rebuild)
        
        self.play(
            Rotate(current1, about_point=ORIGIN, angle=PI),
            Rotate(current2, about_point=ORIGIN, angle=PI),
            run_time=2,
        )

In [ ]:
%%manim -v WARNING -qh Pendulum


# příklad upravený z https://github.com/Matheart/manim-physics
# opět používáme SpaceScene, jelikož animujeme fyzikální interakce
class Pendulum(SpaceScene):
    def construct(self):
        # pozice kuliček pendula
        bob_positions = [RIGHT * 1.5 + UP, RIGHT * 1.5 + UP * 2]
        
        pendulum = MultiPendulum(
            *bob_positions,
            pivot_point=UP,
            bob_style={"color": WHITE, "fill_opacity": 1, "radius": 0.15},
        )
        
        self.make_rigid_body(pendulum.bobs)  # kuličky pendula jsou rigidní
        pendulum.start_swinging()            # a spojené
        
        self.add(pendulum)
        
        # budeme sledovat cestu obou kuliček
        for i, bob in enumerate(pendulum.bobs):
            self.bring_to_back(TracedPath(bob.get_center, stroke_color=DARK_GRAY))
        
        self.wait(12)

### Chemie

In [ ]:
from chanim import *

In [ ]:
%%manim -v WARNING -qh ChanimExample


class ChanimExample(Scene):
    def construct(self):
        # chemická sloučenina
        # interně využívá ChemFigový syntax (https://www.ctan.org/pkg/chemfig)
        chem = ChemWithName(
            "*6((=O)-N(-CH_3)-*5(-N=-N(-CH_3)-=)--(=O)-N(-H_3C)-)",
            "Caffeine"
        )
        
        chem.move_to(ORIGIN)
        
        self.play(chem.creation_anim())